In [1]:

import cv2
import time
import threading
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from flask import Flask, request,render_template,Response,jsonify
import pdfkit
import pickle 
import csv
import mediapipe as mp
import numpy as np
from datetime import datetime
from utils.recorder import StreamParams,Recorder
import pandas as pd
from live_prediction import LivePredictions
import speech_recognition as sr
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
from pathlib import Path
import speech_recognition as sr

In [10]:
config=pdfkit.configuration(wkhtmltopdf=r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe')
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
camera_active = True 

def ajouter_entree(v1, v2, v3):
    with open('static/data3.csv', mode='a', newline='') as file:
        writer = csv.writer(file,delimiter=';')
        writer.writerow([v1, v2, v3])

In [20]:
import cv2
import mediapipe as mp
import pickle
import numpy as np
import pandas as pd
import time
from datetime import datetime

# Initialize MediaPipe components
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Load pre-trained model
with open('body_languagefinal.pkl', 'rb') as f:
    model = pickle.load(f)

def capture_video():
    cap = cv2.VideoCapture(1, cv2.CAP_MSMF)
    
    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    # Get the default frame size from the camera
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the output video file
    video_output = 'output_video.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS) if cap.get(cv2.CAP_PROP_FPS) > 0 else 30  # Use camera FPS or default to 30

    # Initialize the video writer
    video_writer = cv2.VideoWriter(video_output, fourcc, fps, (frame_width, frame_height))
    
    # Initialize holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            
            if not ret:
                print("Error: Failed to grab frame.")
                break

            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        
            
            # Make Detections
            results = holistic.process(image)
            
            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Draw landmarks
            mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                    mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                    )
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                    )
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                    )
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                    )
            
            # Extract and print landmarks
            try:
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                
                face = results.face_landmarks.landmark
                face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
                
                right_hand = results.right_hand_landmarks.landmark if results.right_hand_landmarks else []
                right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
                
                left_hand = results.left_hand_landmarks.landmark if results.left_hand_landmarks else []
                left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
                
                row = pose_row + face_row + right_hand_row + left_hand_row
                
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
                print(body_language_class, body_language_prob)
                
                # Draw text and rectangles on the image
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [frame_width, frame_height]).astype(int))
                
                cv2.rectangle(image, 
                            (coords[0], coords[1]+5), 
                            (coords[0]+len(body_language_class)*20, coords[1]-30), 
                            (245, 117, 16), -1)
                cv2.putText(image, body_language_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                
                # Display Class
                cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Display Probability
                cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2)), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            except:
                pass

            # Write frame to video file
            video_writer.write(image)

            # Display the image
            cv2.imshow('Raw Webcam Feed', image)

            # Check for exit key
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            
    # Release resources
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()


In [ ]:
capture_video()

In [12]:
import threading
# Sous-classe de Thread pour capturer la vidéo
class VideoCaptureThread(threading.Thread):
    def __init__(self):
        super().__init__()
        self.video_result = None

    def run(self):
        self.video_result = capture_video()

    def get_result(self):
        return self.video_result

In [13]:
video_thread = VideoCaptureThread()

In [ ]:
video_thread.start()

In [19]:
import cv2

def main():
    # Initialiser la capture vidéo avec la caméra USB
    # 0 est généralement l'index pour la caméra intégrée, 1 ou plus pour les caméras USB
    cap = cv2.VideoCapture(1)

    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la caméra.")
        return

    print("Appuyez sur 'q' pour quitter la fenêtre de la caméra.")

    while True:
        # Capture une image de la caméra
        ret, frame = cap.read()

        if not ret:
            print("Erreur : Impossible de capturer une image.")
            break

        # Affiche l'image capturée
        cv2.imshow('Flux Vidéo', frame)

        # Quitter la fenêtre en appuyant sur 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Appuyez sur 'q' pour quitter la fenêtre de la caméra.


KeyboardInterrupt: 